# An attempt to hack and prune fasai tabular NN with optuna (For the fun of it)

In [1]:
import numpy as np
import pandas as pd

import os

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold,KFold
from tqdm import tqdm
import numpy as np
import pickle
import optuna

# Metrics for models evaluation
from sklearn.metrics import roc_auc_score
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [1]:
!pip install fastai2 --quiet

In [1]:
!pip install optuna==2.9.1 -U --quiet

In [1]:
from pathlib import Path

input_path = Path('../input/tabular-playground-series-sep-2021/')
TARGET_VAR='claim'
FOLDS=10
X_test = pd.read_feather(f"../input/10-folds-stratified-parquet-feather/test_stratfold.ft")

In [1]:
TARGET_VAR='claim'
FOLDS=10
X= pd.read_feather("../input/10-folds-stratified-parquet-feather/train_stratfold.ft")


In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
num_cols  = [c for c in X.columns if c.startswith("f")] 

X["nan_count"] = X.isnull().sum(axis=1)
X_test["nan_count"] = X_test.isnull().sum(axis=1)
X['std'] = X[num_cols].std(axis=1)
X_test['std'] = X_test[num_cols].std(axis=1)

X['mean'] = X[num_cols].mean(axis=1)
X_test['mean'] = X_test[num_cols].mean(axis=1)

X['kurt'] = X[num_cols].kurtosis(axis=1)
X_test['kurt'] = X_test[num_cols].kurtosis(axis=1)
#scaler = StandardScaler()
useful_features = [c for c in X.columns if c not in ['id','claim' ,'index','kfold']] 
def impute(df):
    for name in useful_features:
        df[name].fillna(df[name].mean(), inplace = True)
    return df    

X=impute(X)
X_test=impute(X_test)

claim = X[TARGET_VAR]
ids=X['id']
X = X.drop(["index","id",TARGET_VAR,"kfold"],axis=1)
scalar = MinMaxScaler()
X[:]= scalar.fit_transform(X) 
ids_test=X_test['id']
X_test=X_test.drop(["index","id"],axis=1)
X_test[:]= scalar.transform(X_test)
#X['min_row'] = X[useful_features].min(axis=1)
#X['max_row'] = X[useful_features].max(axis=1)
#X_test['min_row'] = X_test[useful_features].min(axis=1)
#X_test['max_row'] = X_test[useful_features].min(axis=1)
useful_features = [c for c in X.columns if c not in ['id','claim' ,'index','kfold']] 

X[TARGET_VAR] = claim
X['id'] = ids
X_test['id'] = ids_test

# FAST AI ---->

In [1]:
from fastai.tabular.all import *
from fastai.tabular import * 

processing_funcs = [ Normalize]

test = TabularList.from_df(testdf,  path='.')

data = (TabularList.from_df(traindf, path='.')
            .split_by_idx(list(range(idx, len(traindf))))
            .label_from_df(cols=TARGET_VAR)
            .add_test(test)
            .databunch())

In [1]:
#test_df = TabularPandas(testdf, cat_names=None, cont_names=num_cols, procs=processing_funcs, y_names=TARGET_VAR,
#                                                            #splits= RandomSplitter()(range_of(X)),
#                                                              y_block = CategoryBlock())

nn_df = TabularPandas(X, cat_names=None, cont_names=useful_features, procs=processing_funcs, y_names=TARGET_VAR,
                                                             splits= RandomSplitter()(range_of(X)),
                                                              y_block = CategoryBlock())

In [1]:
nn_df.cont_names

In [1]:
#nn_df.show()

In [1]:
train_dl = nn_df.dataloaders(4096)

In [1]:
#train_dl.show_batch()

In [1]:
dls =  train_dl 

# FASTAI Pruning with Optuna

In [1]:
## Optuna tuning code
# Callback for Pruning.
# Approach from https://github.com/crcrpar/fastai-optuna-rossman/blob/master/optimize_rossman.py
from optuna.integration import FastAIPruningCallback  # v1  
            
def objective(trial):
    # Objective is `exp_rmspe`.
    n_layers = trial.suggest_int('n_layers', 1, 3)
    lr = trial.suggest_loguniform("lr" ,3e-3,3e-2)
    layers = []
    ps = []
    for i in range(n_layers):
        # Number of units
        n_units = trial.suggest_categorical(
            'n_units_layer_{}'.format(i), [36,64,120,240])
        layers.append(n_units)
        # Dropout ratio
        p = trial.suggest_discrete_uniform(
            'dropout_p_layer_{}'.format(i), 0, 1, 0.05)
        ps.append(p)

    #emb_drop = trial.suggest_discrete_uniform('emb_drop', 0, 1, 0.05)
    callback_fns = []
    print("layers",layers," lr ",lr)
    callback_fns.append(
        partial(FastAIPruningCallback, trial=trial, monitor='RocAucBinary')
    )
    learn = tabular_learner(nn_df.dataloaders(), layers=layers, y_range=[0,0.99999999], metrics=[RocAucBinary(),accuracy,error_rate],cbs=[FastAIPruningCallback(trial=trial, monitor="error_rate")])

    res = learn.fit_one_cycle(2, lr, wd=0.2)
    print("AUC : ",learn.recorder.values[-1][2])
    return learn.recorder.values[-1][2]

#study = optuna.create_study(study_name='fastai-optuna-no-pruning', direction='maximize')
#study.optimize(objective, n_trials=40)
#best_trial = study.best_trial

### Selecting number of hidden layers
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

In [1]:
params = {'n_layers': 3, 'lr': 0.005739346471274064, 'n_units_layer_0': 36, 'dropout_p_layer_0': 0.5, 'n_units_layer_1': 36, 'dropout_p_layer_1': 0.9, 'n_units_layer_2': 64, 'dropout_p_layer_2': 0.4}
tab_learn = tabular_learner(nn_df.dataloaders(), y_range=[0,0.99999999],
                            layers=  [36, 36, 64], 
                            metrics=[RocAucBinary(),accuracy,error_rate])#,accuracy, error_rate, Recall(), Precision()])

In [1]:
tab_learn.model

In [1]:
# Learning rate 
tab_learn.lr_find()

# Fit One Cycle
https://fastai1.fast.ai/callbacks.one_cycle.html#What-is-1cycle?

In [1]:
#tab_learn.fit(5)
#tab_learn.fit_one_cycle(8, wd=0.2)
#tab_learn.fit_one_cycle(8,0.006918309628963471, wd=0.2)
tab_learn.fit_one_cycle(7 ,params['lr'], wd=0.2)
# 0.772391
# 0.03630780577659607
#0.772036
# 0.798863 tab_learn.fit_one_cycle(8,  0.005754399299621582, wd=0.2)
# 0.795943
# 0.799196 [] default lauers [200,100]
# 0.800649  [240,62]

In [1]:
tab_learn.recorder.plot_loss()

# Test Set predictions

In [1]:
test_dl = tab_learn.dls.test_dl(X_test)
test_dl.show_batch()

In [1]:
preds, test_labels = tab_learn.get_preds(dl=test_dl)

In [1]:
interpret = ClassificationInterpretation.from_learner(tab_learn)
interpret.plot_confusion_matrix()

In [1]:
interpret.print_classification_report()

In [1]:
final_preds = preds.numpy()[:,1]
#final_preds = np.argmax(final_preds, axis=1)

In [1]:
final_preds

In [1]:
def make_submission(name,final_predictions):
    """Makes submission for testing"""
    sample_submission = pd.read_csv(f"{input_path}/sample_solution.csv")
    try:
        os.remove(f"submission_{name}.csv")
    except (OSError, IOError) as e:    
        #gulp
        print(f"Gulp {name}")
    sample_submission[TARGET_VAR] = final_predictions
    sample_submission.to_csv(f"submission_{name}.csv", index=False)

In [1]:
make_submission("fastai_tab",final_preds) 